In [2]:
import os
from glob import glob
import pandas as pd
from functools import reduce
from xml.etree import ElementTree as et

In [3]:
xml_list = glob('./data_images/*.xml')
xmlfiles = list(map(lambda x : x.replace ('\\', '/'), xml_list))

In [4]:
xmlfiles

['./data_images/cat001.xml',
 './data_images/cat002.xml',
 './data_images/cat003.xml',
 './data_images/cat004.xml',
 './data_images/cat005.xml',
 './data_images/cat006.xml',
 './data_images/cat007.xml',
 './data_images/cat008.xml',
 './data_images/cat009.xml',
 './data_images/cat010.xml',
 './data_images/cat011.xml',
 './data_images/cat012.xml',
 './data_images/cat013.xml',
 './data_images/cat014.xml',
 './data_images/cat015.xml',
 './data_images/cat016.xml',
 './data_images/cat017.xml',
 './data_images/cat018.xml',
 './data_images/cat019.xml',
 './data_images/cat020.xml',
 './data_images/cat021.xml',
 './data_images/cat022.xml',
 './data_images/cat023.xml',
 './data_images/cat024.xml',
 './data_images/cat025.xml',
 './data_images/cat026.xml',
 './data_images/cat027.xml',
 './data_images/cat028.xml',
 './data_images/cat029.xml',
 './data_images/cat030.xml',
 './data_images/cat031.xml',
 './data_images/cat032.xml',
 './data_images/cat033.xml',
 './data_images/cat034.xml',
 './data_image

In [5]:
def extract_text(filename):   
    tree = et.parse(filename)
    root = tree.getroot()
    

    image_name = root.find('filename').text
    width = root.find('size').find('width').text
    height = root.find('size').find('height').text
    objs = root.findall('object')
    parser = []
    for obj in objs:
        name = obj.find('name').text
        bndbox = obj.find('bndbox')
        xmin = bndbox.find('xmin').text
        xmax = bndbox.find('xmax').text
        ymin = bndbox.find('ymin').text
        ymax = bndbox.find('ymax').text
        parser.append ([image_name, width, height, name, xmin, xmax, ymin, ymax])
    
    return parser

In [6]:
parser_all = list(map(extract_text, xmlfiles))

In [7]:
data = reduce(lambda x, y : x+y,parser_all)

In [8]:
data 

[['cat001.jpg', '1000', '688', 'cat', '124', '834', '156', '685'],
 ['cat002.jpg', '1400', '700', 'cat', '390', '1219', '48', '697'],
 ['cat003.jpg', '1080', '1080', 'cat', '140', '685', '1', '1072'],
 ['cat004.jpg', '530', '640', 'cat', '64', '470', '57', '640'],
 ['cat004.jpg', '530', '640', 'cat', '64', '469', '58', '639'],
 ['cat005.jpg', '210', '210', 'cat', '13', '188', '8', '198'],
 ['cat006.jpg', '1400', '845', 'cat', '422', '1259', '13', '840'],
 ['cat007.jpg', '1795', '2397', 'cat', '398', '1608', '266', '2216'],
 ['cat008.jpg', '2938', '4504', 'cat', '883', '2593', '784', '4138'],
 ['cat009.jpg', '700', '467', 'cat', '207', '562', '33', '381'],
 ['cat010.jpg', '1471', '980', 'cat', '44', '1428', '143', '797'],
 ['cat011.jpg', '6000', '4000', 'cat', '354', '4754', '262', '3975'],
 ['cat012.jpeg', '194', '260', 'cat', '45', '183', '27', '239'],
 ['cat013.jpeg', '183', '276', 'cat', '36', '181', '41', '269'],
 ['cat014.jpeg', '225', '225', 'cat', '23', '225', '22', '213'],
 ['c

In [9]:
df = pd.DataFrame(data, columns = ['filename', 'width', 'height', 'name', 'xmin', 'xmax', 'ymin', 'ymax'])

In [10]:
df.head()

,filename,width,height,name,xmin,xmax,ymin,ymax
0,cat001.jpg,1000,688,cat,124,834,156,685
1,cat002.jpg,1400,700,cat,390,1219,48,697
2,cat003.jpg,1080,1080,cat,140,685,1,1072
3,cat004.jpg,530,640,cat,64,470,57,640
4,cat004.jpg,530,640,cat,64,469,58,639


In [11]:
df.shape

(234, 8)

In [12]:
df['name'].value_counts()

name
cat    133
dog    101
Name: count, dtype: int64

In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 234 entries, 0 to 233
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   filename  234 non-null    object
 1   width     234 non-null    object
 2   height    234 non-null    object
 3   name      234 non-null    object
 4   xmin      234 non-null    object
 5   xmax      234 non-null    object
 6   ymin      234 non-null    object
 7   ymax      234 non-null    object
dtypes: object(8)
memory usage: 14.8+ KB


In [14]:
cols = ['width', 'height', 'xmin', 'xmax', 'ymin', 'ymax']
df[cols] = df[cols].astype(int)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 234 entries, 0 to 233
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   filename  234 non-null    object
 1   width     234 non-null    int64 
 2   height    234 non-null    int64 
 3   name      234 non-null    object
 4   xmin      234 non-null    int64 
 5   xmax      234 non-null    int64 
 6   ymin      234 non-null    int64 
 7   ymax      234 non-null    int64 
dtypes: int64(6), object(2)
memory usage: 14.8+ KB


In [15]:
df['center_x'] = ((df['xmax']+df['xmin'])/2)/df['width']
df['center_y'] = ((df['ymax']+df['ymin'])/2)/df['height']
df['w'] = (df['xmax']-df['xmin'])/df['width']
df['h'] = (df['ymax']-df['ymin'])/df['height']

In [16]:
df.head()

,filename,width,height,name,xmin,xmax,ymin,ymax,center_x,center_y,w,h
0,cat001.jpg,1000,688,cat,124,834,156,685,0.479000,0.611192,0.710000,0.768895
1,cat002.jpg,1400,700,cat,390,1219,48,697,0.574643,0.532143,0.592143,0.927143
2,cat003.jpg,1080,1080,cat,140,685,1,1072,0.381944,0.496759,0.504630,0.991667
3,cat004.jpg,530,640,cat,64,470,57,640,0.503774,0.544531,0.766038,0.910937
4,cat004.jpg,530,640,cat,64,469,58,639,0.502830,0.544531,0.764151,0.907813


In [17]:
images = df['filename'].unique()

In [18]:
len(images)

200

In [23]:
img_df = pd.DataFrame(images, columns=['filename'])
img_train = tuple(img_df.sample(frac=0.7)) #uczących 70%

In [24]:
img_test = tuple(img_df.query(f'filename not in {img_train}')['filename']) #testowych 30%

In [29]:
train_df = df.query(f'filename in {img_train}')
test_df = df.query(f'filename in {img_test}')